In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

from keras_preprocessing.image import DataFrameIterator, ImageDataGenerator
import random
import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image

Using TensorFlow backend.


### Base dataframe

In [2]:
pixel_val = 1
np_filenames, img_filenames = [], []
for i in range(20):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'np_filename': np_filenames}).sample(frac=1).reset_index(drop=True)
df['img_path_extra'] = img_filenames
df['regression'] = np.random.randint(1, 10, size=len(df))
df['regression_extra'] = np.random.randint(10, 100, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
print(len(df))
df.head()

20


,img_path,np_filename,img_path_extra,regression,regression_extra,binary,multi_class,multi_label
0,/tmp/3.jpg,/tmp/3.npy,/tmp/0.jpg,1,72,cat,dog,"[cat, dog]"
1,/tmp/8.jpg,/tmp/8.npy,/tmp/1.jpg,3,89,cat,cat,cat
2,/tmp/12.jpg,/tmp/12.npy,/tmp/2.jpg,6,52,cat,horse,[cat]
3,/tmp/11.jpg,/tmp/11.npy,/tmp/3.jpg,2,48,dog,cat,[cat]
4,/tmp/9.jpg,/tmp/9.npy,/tmp/4.jpg,9,75,cat,horse,[cat]


### Multi-output: regression  & regression

In [18]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'regression_extra'],
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/3.jpg', '/tmp/8.jpg', '/tmp/12.jpg', '/tmp/11.jpg', '/tmp/9.jpg', '/tmp/0.jpg', '/tmp/16.jpg', '/tmp/13.jpg', '/tmp/4.jpg', '/tmp/15.jpg', '/tmp/10.jpg', '/tmp/14.jpg', '/tmp/17.jpg', '/tmp/19.jpg', '/tmp/5.jpg', '/tmp/18.jpg', '/tmp/7.jpg', '/tmp/6.jpg', '/tmp/2.jpg', '/tmp/1.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [1, 3, 6, 2, 9, 7, 4, 9, 5, 4, 9, 3, 9, 3, 1, 8, 3, 8, 4, 7]}, {'column': 'regression_extra', 'mode': None, 'values': [72, 89, 52, 48, 75, 14, 22, 24, 16, 81, 74, 48, 66, 27, 21, 40, 77, 74, 18, 48]}]
Epoch 1/1
10/10 [==============================] - 0s 18ms/step - loss: 219068614.1813 - dense_13_loss: 110060208.0000 - dense_14_loss: 109008400.0000


In [19]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['regression', 'regression_extra'],
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'values': ['/tmp/3.npy', '/tmp/8.npy', '/tmp/12.npy', '/tmp/11.npy', '/tmp/9.npy', '/tmp/0.npy', '/tmp/16.npy', '/tmp/13.npy', '/tmp/4.npy', '/tmp/15.npy', '/tmp/10.npy', '/tmp/14.npy', '/tmp/17.npy', '/tmp/19.npy', '/tmp/5.npy', '/tmp/18.npy', '/tmp/7.npy', '/tmp/6.npy', '/tmp/2.npy', '/tmp/1.npy']}]

[{'column': 'regression', 'mode': None, 'values': [1, 3, 6, 2, 9, 7, 4, 9, 5, 4, 9, 3, 9, 3, 1, 8, 3, 8, 4, 7]}, {'column': 'regression_extra', 'mode': None, 'values': [72, 89, 52, 48, 75, 14, 22, 24, 16, 81, 74, 48, 66, 27, 21, 40, 77, 74, 18, 48]}]
Epoch 1/1
10/10 [==============================] - 0s 20ms/step - loss: 477437915.6832 - dense_15_loss: 238082896.0000 - dense_16_loss: 239355008.0000


### Multi-output: regression & binary classification

In [20]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'binary_crossentropy'])
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/3.jpg', '/tmp/8.jpg', '/tmp/12.jpg', '/tmp/11.jpg', '/tmp/9.jpg', '/tmp/0.jpg', '/tmp/16.jpg', '/tmp/13.jpg', '/tmp/4.jpg', '/tmp/15.jpg', '/tmp/10.jpg', '/tmp/14.jpg', '/tmp/17.jpg', '/tmp/19.jpg', '/tmp/5.jpg', '/tmp/18.jpg', '/tmp/7.jpg', '/tmp/6.jpg', '/tmp/2.jpg', '/tmp/1.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [1, 3, 6, 2, 9, 7, 4, 9, 5, 4, 9, 3, 9, 3, 1, 8, 3, 8, 4, 7]}, {'column': 'binary', 'mode': 'sparse', 'values': [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 22ms/step - loss: 129108741.5211 - dense_17_loss: 129103056.0000 - dense_18_loss: 5688.9795


### Multi-output: regression & multi-class classification

In [21]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(3, activation='softmax')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'categorical_crossentropy'])
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/3.jpg', '/tmp/8.jpg', '/tmp/12.jpg', '/tmp/11.jpg', '/tmp/9.jpg', '/tmp/0.jpg', '/tmp/16.jpg', '/tmp/13.jpg', '/tmp/4.jpg', '/tmp/15.jpg', '/tmp/10.jpg', '/tmp/14.jpg', '/tmp/17.jpg', '/tmp/19.jpg', '/tmp/5.jpg', '/tmp/18.jpg', '/tmp/7.jpg', '/tmp/6.jpg', '/tmp/2.jpg', '/tmp/1.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [1, 3, 6, 2, 9, 7, 4, 9, 5, 4, 9, 3, 9, 3, 1, 8, 3, 8, 4, 7]}, {'column': 'multi_class', 'mode': 'categorical', 'values': [1, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, 2, 0, 0, 2, 2, 2, 0, 2, 1], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 29ms/step - loss: 107967513.8389 - dense_19_loss: 107937240.0000 - dense_20_loss: 30275.5352


### Multi-output: multi-class & multi-label classification

In [22]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class', 'multi_label'],
    output_modes={'multi_class': 'categorical', 'multi_label': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(3, activation='softmax')(x)
out_2 = Dense(2, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['categorical_crossentropy', 'binary_crossentropy'])
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/3.jpg', '/tmp/8.jpg', '/tmp/12.jpg', '/tmp/11.jpg', '/tmp/9.jpg', '/tmp/0.jpg', '/tmp/16.jpg', '/tmp/13.jpg', '/tmp/4.jpg', '/tmp/15.jpg', '/tmp/10.jpg', '/tmp/14.jpg', '/tmp/17.jpg', '/tmp/19.jpg', '/tmp/5.jpg', '/tmp/18.jpg', '/tmp/7.jpg', '/tmp/6.jpg', '/tmp/2.jpg', '/tmp/1.jpg']}]
[{'column': 'multi_class', 'mode': 'categorical', 'values': [1, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, 2, 0, 0, 2, 2, 2, 0, 2, 1], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}, {'column': 'multi_label', 'mode': 'categorical', 'values': [[0, 1], 0, [0], [0], [0], [0, 1], [0, 1], 0, 0, 1, [1], 1, [1], [1, 0], 1, 0, [0, 1], 0, [1, 0], [1, 0]], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 34ms/step - loss: 20040.2640 - dense_21_loss: 10399.0762 - dense_22_loss: 9641.1895
